<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Model-Tuning" data-toc-modified-id="Model-Tuning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model Tuning</a></span><ul class="toc-item"><li><span><a href="#Hyperparameters" data-toc-modified-id="Hyperparameters-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hyperparameters</a></span><ul class="toc-item"><li><span><a href="#Difference-from-Parametric-/-Non-Parametric-Models" data-toc-modified-id="Difference-from-Parametric-/-Non-Parametric-Models-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Difference from Parametric / Non-Parametric Models</a></span></li></ul></li><li><span><a href="#Trying-Different-Values" data-toc-modified-id="Trying-Different-Values-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Trying Different Values</a></span><ul class="toc-item"><li><span><a href="#Data-Prep" data-toc-modified-id="Data-Prep-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Data Prep</a></span></li><li><span><a href="#Preparing-the-Test-Set" data-toc-modified-id="Preparing-the-Test-Set-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Preparing the Test Set</a></span></li><li><span><a href="#$k$-Nearest-Neighbors-Model" data-toc-modified-id="$k$-Nearest-Neighbors-Model-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>$k$-Nearest Neighbors Model</a></span><ul class="toc-item"><li><span><a href="#Decreasing-$k$" data-toc-modified-id="Decreasing-$k$-2.2.3.1"><span class="toc-item-num">2.2.3.1&nbsp;&nbsp;</span>Decreasing $k$</a></span></li></ul></li><li><span><a href="#Decision-Tree" data-toc-modified-id="Decision-Tree-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Decision Tree</a></span><ul class="toc-item"><li><span><a href="#Changing-the-branching-criterion" data-toc-modified-id="Changing-the-branching-criterion-2.2.4.1"><span class="toc-item-num">2.2.4.1&nbsp;&nbsp;</span>Changing the branching criterion</a></span></li></ul></li></ul></li><li><span><a href="#Gridsearching" data-toc-modified-id="Gridsearching-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Gridsearching</a></span><ul class="toc-item"><li><span><a href="#GridSearchCV" data-toc-modified-id="GridSearchCV-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span><code>GridSearchCV</code></a></span></li><li><span><a href="#Choice-of-Grid-Values" data-toc-modified-id="Choice-of-Grid-Values-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Choice of Grid Values</a></span></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Exercise</a></span></li><li><span><a href="#GridSearching-a-Pipeline" data-toc-modified-id="GridSearching-a-Pipeline-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>GridSearching a Pipeline</a></span></li><li><span><a href="#Random-Searching" data-toc-modified-id="Random-Searching-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Random Searching</a></span><ul class="toc-item"><li><span><a href="#RandomizedSearchCV-with-LogisticRegression" data-toc-modified-id="RandomizedSearchCV-with-LogisticRegression-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span><code>RandomizedSearchCV</code> with <code>LogisticRegression</code></a></span></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Exercise</a></span></li></ul></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as stats

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Objectives

SWBAT:

- Explain what hyperparameters are
- Describe the purpose of gridsearching
- Implement gridsearching for the purposes of model optimization.

# Model Tuning

## Hyperparameters

Many of the models we have looked at are really *families* of models in the sense that they make use of **hyperparameters**.

Thus for example the $k$-nearest-neighbors algorithm allows us to make:

- a 1-nearest-neighbor model;
- a 2-nearest-neighbors model;
- a 3-nearest-neighbors model;
- etc.

Or, for another example, the decision tree algorithm allows us to make:

- a classifier that branches according to information gain;
- a classifier that branches according to Gini impurity;
- a regressor that branches according to mean squared error;
- etc.

Depending on the sort of problem and data at hand, it is natural to experiment with different values of these hyperparameters to try to improve model performance.

### Difference from Parametric / Non-Parametric Models

Contrast the notion of hyperparameters with the distinction between parametric and non-parametric models.

A linear regression model is parametric in the sense that we start with a given model *form* and we then search for the optimal parameters to fill in that form. But *those* parameters are not the sort we might tweak for the purposes of improving model performance. On the contrary, there is one best set of parameters, and the training of the model is a matter of finding those optimal values.

## Trying Different Values

In [ ]:
penguins = sns.load_dataset('penguins')

penguins.info()

In [ ]:
penguins.head()

### Data Prep

We'll try to predict species given the other columns' values. Let's dummy-out `island` and `sex`:

In [ ]:
penguins.isna().sum().sum()

In [ ]:
penguins = penguins.dropna()

In [ ]:
y = penguins.pop('species')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    penguins, y, test_size=0.5, random_state=42)

In [ ]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

In [ ]:
X_train_nums = X_train.select_dtypes('float64')

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                      index=X_train_nums.index)

In [ ]:
X_train_clean = pd.concat([nums_df, dums_df], axis=1)

In [ ]:
X_train_clean.head()

In [ ]:
knn_model = KNeighborsClassifier()

knn_model.fit(X_train_clean, y_train)

In [ ]:
scores = cross_val_score(estimator=knn_model, X=X_train_clean,
               y=y_train, cv=10)
scores

In [ ]:
np.median(scores)

### Preparing the Test Set

In [ ]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)
test_dums_df = pd.DataFrame(test_dums,
                       columns=ohe.get_feature_names(),
                      index=X_test_cat.index)

In [ ]:
X_test_nums = X_test.select_dtypes('float64')

test_nums = ss.transform(X_test_nums)
test_nums_df = pd.DataFrame(test_nums,
                           index=X_test_nums.index)

In [ ]:
X_test_clean = pd.concat([test_nums_df,
                 test_dums_df], axis=1)

In [ ]:
X_test_clean

### $k$-Nearest Neighbors Model

In [ ]:
knn_model.score(X_test_clean, y_test)

#### Decreasing $k$

In [ ]:
knn5 = KNeighborsClassifier(n_neighbors=3)

knn5.fit(X_train_clean, y_train)

In [ ]:
knn5.score(X_test_clean, y_test)

### Decision Tree

In [ ]:
ct = DecisionTreeClassifier(random_state=10)

ct.fit(X_train_clean, y_train)

In [ ]:
ct.score(X_test_clean, y_test)

#### Changing the branching criterion

In [ ]:
ct = DecisionTreeClassifier(criterion='entropy',
                          random_state=10)

ct.fit(X_train_clean, y_train)

In [ ]:
ct.score(X_test_clean, y_test)

## Gridsearching

It's not a bad idea to experiment with the values of your models' hyperparameters a bit as you're getting a feel for your models' performance. But there are more systematic ways of going about the search for optimal hyperparameters. One method of hyperparameter tuning is **gridsearching**. 

The idea is to build multiple models with different hyperparameter values and then see which one performs the best. The hyperparameters and the values to try form a sort of *grid* along which we are looking for the best performance. For example:


    1           | 'minkowski' | 'uniform'
    3           | 'manhattan' | 'distance'
    5           |
    ______________________________________
    n_neighbors | metric      | weights

Scikit-Learn has a `GridSearchCV` class whose `fit()` method runs this procedure. Note that this can be quite computationally expensive since:

- A model is constructed for each combination of hyperparameter values that we input; and
- Each model is cross-validated.

### `GridSearchCV`

In [ ]:
# Define the parameter grid

grid = {
    'n_neighbors': [1, 3, 5],
    'metric': ['minkowski', 'manhattan'],
    'weights': ['uniform', 'distance']
}

**Question: How many models will we be constructing with this grid?**

In [ ]:
# Initialize the gridsearch object with five-fold cross-validation

gs = GridSearchCV(estimator=knn_model, param_grid=grid, cv=5)

In [ ]:
gs.fit(X_train_clean, y_train)

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
gs.best_estimator_.score(X_test_clean, y_test)

In [ ]:
gs.cv_results_

### Choice of Grid Values

Which values should you pick for your grid? Intuitively, you should try both "large" and "small" values, but of course what counts as large and small will really depend on the type of hyperparameter.

- For a $k$-nearest neighbors model, 1 or 3 would be a small value for the number of neighbors and 15 or 17 would be a large value.
- For a decision tree model, what counts as a small `max_depth` will really depend on the size of your training data. A `max_depth` of 5 would likely have little effect on a very small dataset but, at the same time, it would probably significantly decrease the variance of a model where the dataset is large.
- For a logistic regression's regularization constant, you may want to try a set of values that are exponentially separated, like \[1, 10, 100, 1000\].
- **If a gridsearch finds optimal values at the ends of your hyperparameter ranges, you might try another gridsearch with more extreme values.**

## Exercise

Do a GridSearch on a **decision tree model** of penguin species. What are the optimal values for the hyperparameters you've chosen?

## GridSearching a Pipeline

In [ ]:
ss_pipe = Pipeline(steps=[
    ('ss', StandardScaler())
])
                
ohe_pipe = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first',
                         sparse=False))
])

trans = ColumnTransformer(transformers=[
    ('ss', ss_pipe, X_train_nums.columns),
    ('ohe', ohe_pipe, X_train_cat.columns)
])

In [ ]:
trans.fit(X_train)

In [ ]:
X_train_tr = trans.transform(X_train)

In [ ]:
model_pipe = Pipeline(steps=[
    ('trans', trans),
    ('knn', KNeighborsClassifier())
])

In [ ]:
model_pipe.fit(X_train, y_train)

In [ ]:
model_pipe.score(X_train, y_train)

In [ ]:
pipe_grid = {'knn__n_neighbors': [3, 5, 7], 'knn__p': [1, 2, 3]}

gs_pipe = GridSearchCV(estimator=model_pipe, param_grid=pipe_grid)

In [ ]:
gs_pipe.fit(X_train, y_train)

In [ ]:
gs_pipe.best_params_

## Random Searching

It is also possible to search for good hyperparameter values randomly. This is a nice choice if computation time is an issue or if you are tuning over continuous hyperparameters.

### `RandomizedSearchCV` with `LogisticRegression`

In [ ]:
log_reg_grid = {'C': stats.uniform(loc=0, scale=10),
               'l1_ratio': stats.expon(scale=0.2)}

In [ ]:
rs = RandomizedSearchCV(estimator=LogisticRegression(penalty='elasticnet',
                                                    solver='saga',
                                                    max_iter=1000,
                                                    random_state=42),
                        param_distributions=log_reg_grid,
                       random_state=42)

rs.fit(X_train_clean, y_train)

rs.best_params_

## Exercise

Use a Random Forest Classifier to predict the category of price range for the phones in this dataset. Try tuning some hyperparameters using GridSearch, and then write up a short paragraph about your findings.

In [ ]:
phones_train = pd.read_csv('data/train.csv')

phones_test = pd.read_csv('data/test.csv')